In [1]:
import arcpy
import requests
import os
import zipfile
import io

In [12]:
# Set variable for data directory and set workspace as directory
directory = r'C:\Users\cason\Desktop\GIS5571\FinalProject\data'
arcpy.env.workspace = directory

# Post requests for input data and extract zipfiles

In [6]:
# Hennepin County racial covenant data from Data Repository for U of M
rclink = r'https://conservancy.umn.edu/bitstream/handle/11299/217209/Hennepin_County_Racial_Covenants.zip?sequence=9&isAllowed=y'
rcpostrequest = requests.post(rclink)
rccontent = rcpostrequest.content
rczip = zipfile.ZipFile(io.BytesIO(rccontent))
rcshp = rczip.extractall(directory)

In [7]:
# American Community Survey (ACS) 5-Year Summary File from MN Geo Commons
acslink = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census_acs/shp_society_census_acs.zip'
acspostrequest = requests.post(acslink)
acscontent = acspostrequest.content
acszip = zipfile.ZipFile(io.BytesIO(acscontent))
acstable = acszip.extractall(directory)

In [8]:
# 2020 Census boundary shapefile from MN Geo Commons
acsgeolink = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census2020tiger/shp_society_census2020tiger.zip'
acsgeopostrequest = requests.post(acsgeolink)
acsgeocontent = acsgeopostrequest.content
acsgeozip = zipfile.ZipFile(io.BytesIO(acsgeocontent))
acsgeoshp = acsgeozip.extractall(directory)

In [9]:
# 2015 Twin Cities Metropolitan Area Urban Tree Canopy Assessment from Data Repository for U of M
canopylink = r'https://conservancy.umn.edu/bitstream/handle/11299/183470/TCMA_UrbanTreeCanopy2015.zip?sequence=6&isAllowed=y'
canopypostrequest = requests.post(canopylink)
canopycontent = canopypostrequest.content
canopyzip = zipfile.ZipFile(io.BytesIO(canopycontent))
canopyraster = canopyzip.extractall(directory)

In [10]:
# MN County boundary shapefile from MN Geo Commons
countieslink = r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip'
countiespostrequest = requests.post(countieslink)
countiescontent = countiespostrequest.content
countieszip = zipfile.ZipFile(io.BytesIO(countiescontent))
countiesshp = countieszip.extractall(directory)

In [13]:
# Set variables for input data from extracted zipfiles
rc = 'Hennepin_County_Racial_Covenants.shp'
acs = 'CensusACSBlockGroup.dbf'
blockgroups = 'Census2020TigerCollarBlockGroup.shp'
canopy = 'tcma_lc_treecanopy_v1.tif'
counties = 'mn_county_boundaries.shp'

# Racial Covenants Data Preparation

In [14]:
# Calculate RC field as 1 for all racial covenant features
arcpy.management.CalculateField(rc, 
                                "RC", 
                                "1", 
                                "PYTHON3", 
                                '', 
                                "TEXT", 
                                "NO_ENFORCE_DOMAINS")

<Result 'C:\\Users\\cason\\Desktop\\GIS5571\\FinalProject\\data\\Hennepin_County_Racial_Covenants.shp'>

In [15]:
# Create shapefile for Hennepin County study extent boundary
hennbound = arcpy.conversion.FeatureClassToFeatureClass(counties, 
                                                        directory, 
                                                        "HennepinBound.shp", 
                                                        "CTY_NAME = 'Hennepin'", 
                                                        r'AREA "AREA" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,AREA,-1,-1;PERIMETER "PERIMETER" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,PERIMETER,-1,-1;CTYONLY_ "CTYONLY_" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,CTYONLY_,-1,-1;CTYONLY_ID "CTYONLY_ID" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,CTYONLY_ID,-1,-1;COUN "COUN" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,COUN,-1,-1;CTY_NAME "CTY_NAME" true true false 20 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,CTY_NAME,0,20;CTY_ABBR "CTY_ABBR" true true false 4 Text 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,CTY_ABBR,0,4;CTY_FIPS "CTY_FIPS" true true false 4 Short 0 4,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,CTY_FIPS,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,C:\Users\cason\Desktop\GIS5571\FinalProject\data\mn_county_boundaries.shp,Shape_Area,-1,-1', 
                                                        '')

In [22]:
# Combine racial covenant shapefile and Hennepin County boundary shapefile for raster conversion
rchennshp = arcpy.analysis.Union([rc, hennbound], 
                                 directory + r'\rc_henn', 
                                 "ALL", 
                                 None, 
                                 "GAPS")

In [23]:
# Convert Hennepin racial covenants shapefile to raster layer with same 1 meter resolution as tree canopy raster
rcraster = arcpy.conversion.PolygonToRaster(rchennshp, 
                                            "RC", 
                                            directory + r'\rc_henn_rast', 
                                            "CELL_CENTER", 
                                            "NONE", 
                                            canopy, 
                                            "BUILD")

In [ ]:
# Reclassify racial covenants raster
rcreclass = arcpy.sa.Reclassify("rc_henn_rast", 
                                "RC", 
                                "0 10;1 1;NODATA 10", 
                                "DATA"); 

rcreclass.save(directory + r'\rc_reclass')

# ACS Poverty Data Preparation

In [ ]:
# Join ACS block groups table with block group boundaries shapefile
acsjoin = arcpy.management.AddJoin(blockgroups, 
                                   "GEOID20", 
                                   acs, 
                                   "GEOID2", 
                                   "KEEP_ALL", 
                                   "INDEX_JOIN_FIELDS")

In [ ]:
# Clip ACS block group shapefile to Hennepin County study extent boundary
acshenn = arcpy.analysis.Clip(acsjoin, 
                              hennbound, 
                              directory + r'\ACSHenn', 
                              None)

In [ ]:
# Convert ACS block group shapefile to raster using POV185RATE field with same 1 meter resolution as tree canopy raster
acsraster = arcpy.conversion.PolygonToRaster(acshenn, 
                                             "CensusACSBlockGroup_POV185RATE", 
                                             directory + r'\ACSHenn_POV185RATE_Raster', 
                                             "CELL_CENTER", 
                                             "NONE", 
                                             canopy, 
                                             "BUILD")

In [ ]:
# Reclassify ACS raster layer to highlight areas of significant poverty
acsreclass = arcpy.sa.Reclassify(acsraster, 
                                 "VALUE", 
                                 "0 0.082353 10;0.082353 0.203922 3;0.203922 0.384314 3;0.384314 0.623529 2;0.623529 1 1;NODATA 10", 
                                 "DATA"); 

acsreclass.save(directory + r'\ACSHenn_Reclass")

# Canopy Data Preparation

In [ ]:
# Extract canopy raster by Hennepin County study extent boundary mask
canopyhenn = arcpy.sa.ExtractByMask(canopy, 
                                    hennbound, 
                                    "INSIDE", 
                                    '439324.861 4959173.419 486049.45 5010478.118 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'); 

canopyhenn.save(directory + r'\TreeCanopyHenn')

In [ ]:
# Reclassify canopy raster to highlight areas suitable for increased tree canopy
canopyreclass = arcpy.sa.Reclassify(canopyhenn, 
                                    "Class_Name", 
                                    "' ' 10;Grass/Shrub 1;'Bare Soil' 1;Buildings 10;'Roads/Paved Surfaces' 10;Lakes/Ponds 10;'Deciduous Tree Canopy' 10;'Coniferous Tree Canopy' 10;Agriculture 1;'Emergent Wetland' 10;'Forested/Shrub Wetland' 10;River 10;Extraction 10;NODATA 10", 
                                    "DATA"); 

canopyreclass.save(directory + r'\TreeCanopyReclass')

# Produce various Weighted Overlay results

In [ ]:
# Tree Canopy: 40%
# ACS: 30%
# Racial Covenants: 30%
overlay1 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\FinalProject\data\Tree_Canopy_Reclass' 40 'Value' (1 1; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\ACS_Henn_Reclass' 30 'Value' (1 1; 2 2; 3 3; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\RacialCovReclass' 30 'Value' (1 1; 10 10; NODATA 10));1 10 1"); 

overlay1.save(directory + r'\Overlay1')

In [ ]:
# Tree Canopy: 50%
# ACS: 20%
# Racial Covenants: 30%
overlay2 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\FinalProject\data\Tree_Canopy_Reclass' 50 'Value' (1 1; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\ACS_Henn_Reclass' 20 'Value' (1 1; 2 2; 3 3; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\RacialCovReclass' 30 'Value' (1 1; 10 10; NODATA 10));1 10 1"); 

overlay2.save(directory + r'\Overlay2')

In [ ]:
# Tree Canopy: 50%
# ACS: 30%
# Racial Covenants: 20%
overlay3 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\FinalProject\data\Tree_Canopy_Reclass' 50 'Value' (1 1; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\ACS_Henn_Reclass' 30 'Value' (1 1; 2 2; 3 3; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\RacialCovReclass' 20 'Value' (1 1; 10 10; NODATA 10));1 10 1"); 

overlay3.save(directory + r'\Overlay3')

In [ ]:
# Tree Canopy: 20%
# ACS: 40%
# Racial Covenants: 40%
overlay4 = arcpy.sa.WeightedOverlay(r"('C:\Users\cason\Desktop\GIS5571\FinalProject\data\Tree_Canopy_Reclass' 20 'Value' (1 1; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\ACS_Henn_Reclass' 40 'Value' (1 1; 2 2; 3 3; 10 10; NODATA 10); 'C:\Users\cason\Desktop\GIS5571\FinalProject\data\RacialCovReclass' 40 'Value' (1 1; 10 10; NODATA 10));1 10 1"); 

overlay4.save(directory + r'\Overlay4')